In [1]:
'''
找就位道、计算倒凹、填倒凹
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/9月下旬项目重开"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

obj_file_path = "肩台外侧点-0715/1/1_1.obj"
txt_file_path = "多分类预处理0929/预处理结果文档/3.txt"

当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\9月下旬项目重开


In [2]:
# 准备数据
import numpy as np

def load_labeled_point_cloud(file_path):
    '''读取点云数据和标签'''
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)


In [3]:
'''
    # 1. 读取点云数据
    vertices, labels = load_labeled_point_cloud(txt_file_path)
    crown_idx = [idx for idx in range(len(labels)) if labels[idx]!=1]
    crown_pts = vertices[crown_idx]
'''

'\n    # 1. 读取点云数据\n    vertices, labels = load_labeled_point_cloud(txt_file_path)\n    crown_idx = [idx for idx in range(len(labels)) if labels[idx]!=1]\n    crown_pts = vertices[crown_idx]\n'

In [4]:
import numpy as np
import open3d as o3d
from scipy.spatial import ConvexHull

def estimate_normals(points, k=30):
    """估计点云的法向量，使用open3d"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=k))
    normals = np.asarray(pcd.normals)
    return normals

def sample_directions_around_y_axis(n_samples, half_angle_degrees=20):
    """以y轴为中心，在半角20度锥形范围内均匀采样方向向量"""
    half_angle = np.radians(half_angle_degrees)
    
    directions = []
    for _ in range(n_samples):
        phi = np.random.uniform(0, 2 * np.pi)
        theta = np.random.uniform(0, half_angle)
        
        x = np.sin(theta) * np.cos(phi)
        z = np.sin(theta) * np.sin(phi)
        y = np.cos(theta)
        
        direction = np.array([x, y, z])
        directions.append(direction)
    
    return np.array(directions)

def project_points_onto_plane(points, direction):
    """将点云投影到与给定方向垂直的平面上"""
    # 计算投影矩阵，去除沿着就位道方向的分量
    direction_normalized = direction / np.linalg.norm(direction)
    projection_matrix = np.eye(3) - np.outer(direction_normalized, direction_normalized)
    projected_points = points @ projection_matrix.T
    return projected_points

def detect_undercut(points, normals, direction):
    """使用几何遮挡检测倒凹区域"""
    projected_points = project_points_onto_plane(points, direction)
    
    # 构建凸包来确定包裹点集的区域
    hull = ConvexHull(projected_points[:, [0, 2]])  # 投影到x-z平面
    hull_points = projected_points[hull.vertices]

    # 对比点云，判断哪些点处于倒凹中
    undercut_indices = []
    for i, point in enumerate(projected_points):
        if point[1] < np.min(hull_points[:, 1]):  # 判断是否被其他点遮挡
            undercut_indices.append(i)

    undercut_points = points[undercut_indices]
    return undercut_points

def compute_volume_mesh(points):
    """通过三角化计算点云的体积"""
    if len(points) < 4:
        return 0
    hull = ConvexHull(points)
    return hull.volume

def find_optimal_insertion_path(points, normals, n_directions=500, half_angle_degrees=20):
    """寻找使倒凹体积最小的就位道方向"""
    directions = sample_directions_around_y_axis(n_directions, half_angle_degrees)
    
    min_volume = np.inf
    optimal_direction = None
    optimal_undercut_points = None

    for direction in directions:
        undercut_points = detect_undercut(points, normals, direction)
        undercut_volume = compute_volume_mesh(undercut_points)

        if undercut_volume < min_volume:
            min_volume = undercut_volume
            optimal_direction = direction
            optimal_undercut_points = undercut_points

    return optimal_direction, optimal_undercut_points, min_volume

def create_dashed_line(start_point, direction, length=5.0, segments=20, color=[0, 0, 0], line_width=0.02):
    """创建加粗的黑色虚线，表示就位道方向"""
    end_point = start_point + length * direction
    points = []
    lines = []
    
    for i in range(segments):
        t1 = i / segments
        t2 = (i + 0.5) / segments
        
        segment_start = start_point * (1 - t1) + end_point * t1
        segment_end = start_point * (1 - t2) + end_point * t2
        
        points.append(segment_start)
        points.append(segment_end)
        lines.append([2 * i, 2 * i + 1])

    # 将虚线的每个点和线存入 LineSet
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector([color for _ in lines])

    return line_set

def visualize_point_cloud(points, normals=None, undercut_points=None, optimal_direction=None):
    """使用open3d可视化点云，点云为浅蓝色，倒凹区域为红色，显示就位道方向"""
    # 创建点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # 设置原始点云颜色为浅蓝色
    pcd.paint_uniform_color([0.6, 0.8, 1.0])  # 浅蓝色
    
    geometries = [pcd]
    
    # 倒凹区域设置为红色
    if undercut_points is not None and len(undercut_points) > 0:
        undercut_pcd = o3d.geometry.PointCloud()
        undercut_pcd.points = o3d.utility.Vector3dVector(undercut_points)
        undercut_pcd.paint_uniform_color([1, 0, 0])  # 红色
        geometries.append(undercut_pcd)
    
    # 绘制就位道方向（加粗黑色虚线）
    if optimal_direction is not None:
        center_point = points.mean(axis=0)  # 计算点云中心
        dashed_line = create_dashed_line(center_point, optimal_direction, length=5.0, segments=20, color=[0, 0, 0], line_width=0.02)
        geometries.append(dashed_line)
    
    # 可视化
    o3d.visualization.draw_geometries(geometries)

# 主函数
if __name__ == "__main__":
    # 1. 读取点云数据
    vertices, labels = load_labeled_point_cloud(txt_file_path)
    crown_idx = [idx for idx in range(len(labels)) if labels[idx]!=1]
    crown_pts = vertices[crown_idx]

    # 2. 估计法向量
    normals = estimate_normals(crown_pts)

    # 3. 寻找最佳就位道方向（限制在半角20度内）
    optimal_direction, undercut_points, undercut_volume = find_optimal_insertion_path(crown_pts, normals, n_directions=500, half_angle_degrees=20)

    # 4. 输出结果
    print(f"Optimal Insertion Direction: {optimal_direction}")
    print(f"Undercut Volume: {undercut_volume}")

    # 5. 可视化点云（浅蓝色）、倒凹区域（红色）和就位道方向（加粗黑色虚线）
    visualize_point_cloud(crown_pts, normals=normals, undercut_points=undercut_points, optimal_direction=optimal_direction)


Optimal Insertion Direction: [-0.06225292  0.99804112  0.00620444]
Undercut Volume: 0
